<a href="https://colab.research.google.com/github/juliovicenzi/Benchmarks/blob/master/axur.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Axur selection process


In [5]:
%autosave 60

Autosaving every 60 seconds


In [6]:
!pip install transformers=='4.49.0' datasets=='3.4.1' peft=='0.14.0' evaluate=='0.4.0' bert_score=='0.3.13'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0

## 1. Data collection

In [7]:
from datasets import load_dataset, DatasetDict

RANDOM_SEED=42

def load_alpaca_dataset(
        num_samples: int = 6000,
        test_size: float = 0.5,
        seed: int = RANDOM_SEED
    ) -> DatasetDict:
    """Reads the alpaca dataset from huggingface,
    and splits it into train and test.

    Args:
        num_samples: the number of rows to retrieve from the dataset
        test_size: the train/test split. Must be in [0,1]
        seed: random seed to use during shuffling
    Returns:
        the downloaded dataset split into train and test
    """
    alpaca_data = load_dataset(
        "tatsu-lab/alpaca",
        # alpaca does not contain a test split by default
        split=f"train[:{num_samples}]"
    )

    shuffled_split_data = alpaca_data.train_test_split(
        test_size=0.5,
        shuffle=True,
        seed=RANDOM_SEED
    )
    return shuffled_split_data

data = load_alpaca_dataset()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.47k [00:00<?, ?B/s]

(…)-00000-of-00001-a09b74b3ef9c3b56.parquet:   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [8]:
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 3000
    })
})

Next, we will retrieve gpt2-small model from huggingface, and test it on a random prompt.

In [9]:
from transformers import AutoTokenizer, TrainingArguments, Trainer, GPT2LMHeadModel, GPT2Tokenizer, set_seed

set_seed(RANDOM_SEED)

model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model.pad_token_id = tokenizer.eos_token
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['What is the capital of Germany?\n\nThe capital of Germany is Berlin.\n\nThe capital of Germany is Berlin.\n\n']

In [12]:
prompt = tokenizer("What is the capital of Germany?", return_tensors='pt', padding=True, padding_side='left', truncation=True)
res = model.generate(**prompt)
tokenizer.batch_decode(res)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['What is the capital of Germany?\n\nThe capital of Germany is Berlin.\n\nThe capital of Germany is Berlin.\n\n']

In [13]:
prompt

{'input_ids': tensor([[2061,  318,  262, 3139,  286, 4486,   30]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [48]:

from functools import partial
from datasets import Features, Value, Sequence

def create_prompt(instruction: str, input: str) -> str:
    return f"""Instruction: {instruction}
    Input: {input}
    Response:
    """

# create wrapper function to avoid repeating tokenizer parameters
get_tokens = partial(tokenizer,
    padding="max_length",
    max_length=512,
    truncation=True,
    padding_side="left",
)

def preprocess_sample(sample: dict) -> dict:
    prompt = create_prompt(sample["instruction"], sample["input"])
    tokenized_prompt = get_tokens(sample["instruction"], sample["input"])
    tokenized_response = get_tokens(sample["output"])
    # ensure all tensors are using the same dtypes
    return {
        "inputs": {
            "input_ids": [tokenized_prompt["input_ids"]],
            "attention_mask": [tokenized_prompt["attention_mask"]]
        },
        "labels": {
            "input_ids": [tokenized_response["input_ids"]],
            "attention_mask": [tokenized_response["attention_mask"]]
        }
    }

def preprocess_dataset(data: DatasetDict) -> DatasetDict:
    features = Features(
        {
            "inputs": {
                "input_ids": Sequence(Sequence(Value("int32"))),
                "attention_mask": Sequence(Sequence(Value("int32")))
            },
            "labels": {
                "input_ids": Sequence(Sequence(Value("int32"))),
                "attention_mask": Sequence(Sequence(Value("int32")))
            }
        }
    )
    processed_data = data.map(
        preprocess_sample,
        remove_columns=data.column_names,
        features=features,
    )
    processed_data.set_format(type="torch")
    return processed_data

tokenized_train = preprocess_dataset(data["train"])
tokenized_test = preprocess_dataset(data["test"])

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [45]:
tokenized_train["inputs"][0]

{'input_ids': tensor([[50256, 50256, 50256,  ...,    39,   521,  8132]]),
 'attention_mask': tensor([[0, 0, 0,  ..., 1, 1, 1]])}

In [64]:
model.generate(**tokenized_train["labels"][0], pad_token_id=tokenizer.pad_token_id, max_new_tokens=200)

tensor([[50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 5

In [ ]:
def model_predict(sample) -> dict
    pass

tokenized_test.map(lambda x: {"predictions": tokenizer.batch_decode(model.generate(**x["inputs"], pad_token_id=tokenizer.pad_token_id, max_new_tokens=500))})

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [18]:
tokenized_test[0]

{'input_ids': tensor([30003,  6525,   262,  6827,   287,   262,   717,    12,  6259,   966,
           286,  1570,    13,  1544,   373,   925,   284,  5806,   262,  6576,
            13]),
 'labels': tensor([  40,  373,  925,  284, 5806,  262, 6576,   13])}

In [ ]:
# add evaluation: run each dataset example using pipeline
# add metrics: see what's relevat
# add mlflow? just because if I have the time

In [ ]:
from evaluate import load

def evaluate_model(predictions: list, references: list) -> dict:
    bertscore = load("bertscore")
    perplexity = load("perplexity", module_type="metric")

    bs = bertscore.compute(predictions=predictions, references=references, lang="en")
    perp = perplexity.compute(predictions=predictions, lang="en")
    return {"bertscore": bs, "perplexity": perp}

In [ ]:
import mlflow

mlflow.transformers.autolog()

In [ ]:
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.notebook import tqdm

with mlflow.start_run() as run:
    tokenized_test.set_format(type = "torch")
    predicted = tokenized_test.map(lambda x: {"prediction": model.generate(x["prompt"])})
    references = list(data["test"]["output"])
    nft_res = evaluate_model(predictions=predictions, references=references)


In [51]:
from peft import LoraConfig, get_peft_model, TaskType
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)
ft_model = get_peft_model(model, lora_config)
ft_model.print_trainable_parameters()

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
import evaluate
evaluate.list_evaluation_modules(
  module_type="comparison",
  include_community=False,
  with_details=True
)


In [53]:
from transformers import TrainingArguments, Trainer


def train_model(model, train_data, eval_data):
    # Define training arguments
    training_args = TrainingArguments(
        output_dir='./model',
        overwrite_output_dir=True,
        num_train_epochs=1,
        eval_strategy="steps",
        eval_steps=100,
        save_steps=500,
        logging_steps=100,
        logging_dir='./logs',
    )

    # Create a Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_data,
        eval_dataset=val_data,
    )

    # Fine-tune the model
    trainer.train()

train_model()

In [ ]:
def fine_tune_model(model, train_data, eval_data, epochs: int, learning_rate: float):
    pass

# 3. How would you group questions from the train and test datasets that deal with similar subjects?

First, we can use sentence embedding to get vector representations of the questions.

From the embedding, we can perform similarity grouping either using *similarity search*, like [FASS](https://ai.meta.com/tools/faiss/).

A more traditional but apporach could make use K-means, or any other clustering algorithm to find similarities in the questions based on their sentence embeddings.